In [466]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Boolinger Band

In [467]:


# Load the stock price data into a pandas DataFrame
df = pd.read_csv('./coin_data.csv', index_col=['Symbol', 'Date'], parse_dates=True)




Calculate Boolinger Bands

In [468]:
# Define a function to calculate the Bollinger Bands
def calculate_bollinger_bands(group):

    window = 3
    num_std = 1
    group = group.sort_values(by='Date', ascending=True)

    # Calculate the rolling mean and standard deviation
    rolling_mean = group['Close'].rolling(window=window).mean()
    rolling_std = group['Close'].rolling(window=window).std()

    # Calculate the upper and lower Bollinger Bands
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)

    # Add the Bollinger Bands to the DataFrame
    group['Upper'] = upper_band
    group['Lower'] = lower_band

    # Identify overbought and oversold conditions
    group['Overbought'] = np.where(group['Close'] > group['Upper'], 1, 0)
    group['Oversold'] = np.where(group['Close'] < group['Lower'], 1, 0)

    # Spot trend reversals
    group['Trend'] = np.where(group['Close'] > rolling_mean, 1, -1)
    group['Reversal'] = np.where((group['Trend'].shift(1) == 1) & (group['Trend'] == -1), 1, 0)

    # Identify potential breakouts
    group['Breakout'] = np.where((group['Close'] > group['Upper'].shift(1)) & (group['Close'].shift(1) < group['Upper'].shift(1)), 1, 
                              np.where((group['Close'] < group['Lower'].shift(1)) & (group['Close'].shift(1) > group['Lower'].shift(1)), 0, 0))

    group.drop(["Trend"], axis=1, inplace=True)

    return group

EMA Function

In [469]:
# Define a function to calculate the EMA and check for a cross above/below
def calculate_ema(no_of_days, group):
    group = group.sort_values(by='Date', ascending=True)

    # Calculate the EMA
    ema_col_name = f'EMA{no_of_days}'
    group[ema_col_name] = group['Close'].ewm(span=no_of_days, adjust=False).mean()

    # Check for a cross above the EMA
    cross_above_col_name = f'CrossAboveEMA{no_of_days}'
    group[cross_above_col_name] = np.where((group['Close'].shift(1) < group[ema_col_name].shift(1)) & (group['Close'] > group[ema_col_name]), 1, 0)

    # Check for a cross below the EMA
    cross_below_col_name = f'CrossBelowEMA{no_of_days}'
    group[cross_below_col_name] = np.where((group['Close'].shift(1) > group[ema_col_name].shift(1)) & (group['Close'] < group[ema_col_name]), 1, 0)

    # Check if the Close is above the EMA
    close_above_ema_col_name = f'CloseAboveEMA{no_of_days}'
    group[close_above_ema_col_name] = np.where(group['Close'] > group[ema_col_name], 1, 0)

    # Check if the Close is below the EMA
    close_below_ema_col_name = f'CloseBelowEMA{no_of_days}'
    group[close_below_ema_col_name] = np.where(group['Close'] < group[ema_col_name], 1, 0)

    return group

Calculate Simple moving average

In [470]:
def calculate_sma(window_size, group):
    group = group.sort_values(by='Date', ascending=True)

    # Calculate the SMA
    sma_col_name = f'SMA{window_size}'
    group[sma_col_name] = group['Close'].rolling(window=window_size).mean()

    # Check if the Close is above the SMA
    close_above_sma_col_name = f'CloseAboveSMA{window_size}'
    group[close_above_sma_col_name] = np.where(group['Close'] > group[sma_col_name], 1, 0)

    # Check if the Close is below the SMA
    close_below_sma_col_name = f'CloseBelowSMA{window_size}'
    group[close_below_sma_col_name] = np.where(group['Close'] < group[sma_col_name], 1, 0)

    return group

In [471]:
def calculate_avg_price(window_size, group):
    group = group.sort_values(by='Date', ascending=True)

    # Calculate the average price for the next X days
    avg_price_col_name = f'AvgPriceNext{window_size}Days'
    group[avg_price_col_name] = group['Close'].rolling(window=window_size).mean().shift(-window_size)

    return group

In [472]:
def calculate_macd_signals(group):
    group = group.sort_values(by='Date', ascending=True)

    # Calculate the MACD
    ema_a = group['Close'].ewm(span=2, adjust=False).mean()
    ema_b = group['Close'].ewm(span=4, adjust=False).mean()
    macd = ema_a - ema_b

    # Calculate the signal line
    signal_line = macd.ewm(span=1, adjust=False).mean()

    # Calculate the histogram
    histogram = macd - signal_line

    # Add the MACD, signal line, and histogram to the DataFrame
    group['MACD'] = macd
    group['SignalLine'] = signal_line
    group['Histogram'] = histogram

    # Generate MACD signals
    group['MACD_Crossover'] = np.where((macd > signal_line) & (macd.shift(1) < signal_line.shift(1)), 1, 0)

    group['Zero_Line_Crossover'] = np.where(macd > 0, 1, 0)
    # group['Zero_Line_Crossover'] = np.where(macd < 0, 0, group['Zero_Line_Crossover'])
    group['Divergence'] = np.where((macd.diff() > 0) & (group['Close'].diff() < 0), 1,
                                   np.where((macd.diff() < 0) & (group['Close'].diff() > 0), 0, 0))
    group['Histogram_Expansion'] = np.where(histogram.diff() > 0, 1, 0)
    group['Histogram_Contraction'] = np.where(histogram.diff() < 0, 1, 0)

    return group

In [473]:
def calculate_rsi(group, window):
    group = group.sort_values(by='Date', ascending=True)

    # Calculate price difference
    price_diff = group['Close'].diff()

    # Calculate upward and downward price changes
    up_prices = price_diff.where(price_diff > 0, 0)
    down_prices = -price_diff.where(price_diff < 0, 0)

    # Calculate average gains and losses
    avg_gain = up_prices.rolling(window).mean()
    avg_loss = down_prices.rolling(window).mean()

    # Calculate relative strength (RS)
    rs = avg_gain / avg_loss

    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))

    # Add RSI to the DataFrame
    group['RSI'] = rsi

    # Assuming RSI values are already calculated and available in the DataFrame
    group['RSI_Buy_Signal'] = np.where(group['RSI'] < 30, 1, 0)
    group['RSI_Sell_Signal'] = np.where(group['RSI'] > 70, 1, 0)

    return group

Calculate the crossover from two ema periods

In [474]:
def calculate_ema_crossover(period1, period2, group):
    group = group.sort_values(by='Date', ascending=True)

    # Calculate the EMA for the specified periods
    ema_col_name1 = f'EMA{period1}'
    ema_col_name2 = f'EMA{period2}'
    group[ema_col_name1] = group['Close'].ewm(span=period1, adjust=False).mean()
    group[ema_col_name2] = group['Close'].ewm(span=period2, adjust=False).mean()

    # Generate the crossover signal
    crossover_col_name = f'CrossAboveEMA{period1}EMA{period2}'
    group[crossover_col_name] = np.where((group[ema_col_name1].shift(1) < group[ema_col_name2].shift(1)) & (group[ema_col_name1] > group[ema_col_name2]), 1, 0)

    return group

In [475]:
def calculate_obv_signals(group):
    group = group.sort_values(by='Date', ascending=True)

    # Calculate the OBV
    group['OBV'] = 0
    group.loc[group['Close'] > group['Close'].shift(), 'OBV'] = group['Volume']
    group.loc[group['Close'] < group['Close'].shift(), 'OBV'] = -group['Volume']
    group['OBV'] = group['OBV'].cumsum()

    # Generate OBV signals
    group['OBV_Buy_Signal'] = np.where((group['OBV'].diff() > 0) & (group['OBV'].shift(1) < 0), 1, 0)
    group['OBV_Sell_Signal'] = np.where((group['OBV'].diff() < 0) & (group['OBV'].shift(1) > 0), 1, 0)

    return group

In [ ]:
def calculate_7dayPrice(group):
 
 # Calculate the price difference between the current day and 7 days later
    group['Price7DaysLater'] = group['close'].shift(-7) 
    group['PriceDiffResult'] = group['close'].shift(-7) - group['close']
    # Compare the price difference with a threshold value
    group['Trend7Days'] = np.where(group['PriceDiffResult'] >= 0 , 1, 0)
    return group 

In [476]:

grouped = df.groupby(level='Symbol')
pd.set_option('display.max_columns', 10)
results = pd.DataFrame()



# comment this line for handle all coins
coins_of_interest = ['btc']

# Loop through each group and perform the analysis for coins of interest
for name, group in grouped:
    # Check if the coin is in the coins of interest
    if name.lower() not in coins_of_interest:
        continue
    
    print(f"Analysis for {name}:")
    group = group.sort_values(by='Date', ascending=True)

    group = calculate_bollinger_bands(group)
    group = calculate_ema(100, group)
    group = calculate_ema(180, group)
    group = calculate_ema(50, group)
    group = calculate_ema(20, group)
    group = calculate_ema(10, group)
    group = calculate_ema(3, group)
    group = calculate_ema(5, group)
    group = calculate_ema(7, group)
    group = calculate_sma(10, group)
    group = calculate_sma(3, group)
    group = calculate_sma(5, group)
    group = calculate_sma(7, group)
    group = calculate_sma(20, group)
    group = calculate_sma(50, group)
    group = calculate_sma(100, group)
    group = calculate_sma(180, group)
    group = calculate_macd_signals(group)
    group = calculate_rsi(group,5)
    group = calculate_obv_signals(group)
    group= calculate_avg_price(3,group)
    group= calculate_avg_price(5,group)
    group= calculate_avg_price(7,group)
    group= calculate_ema_crossover(10,20,group)
    group= calculate_ema_crossover(20,50,group)





    results = pd.concat([results, group])

print(results)

Analysis for ada:
Analysis for atom:
Analysis for bnb:
Analysis for btc:
Analysis for dot:
Analysis for eth:
Analysis for fil:
Analysis for ldo:
Analysis for ltc:
Analysis for matic:
Analysis for sol:
Analysis for xrp:
                               Close        Volume     Upper     Lower  \
Symbol Date                                                              
ada    2017-10-18 08:00:00  0.026845  2.351678e+06       NaN       NaN   
       2017-10-19 08:00:00  0.026830  2.815156e+06       NaN       NaN   
       2017-10-20 08:00:00  0.030300  8.883473e+06  0.029990  0.025993   
       2017-10-21 08:00:00  0.028588  5.308857e+06  0.030307  0.026838   
       2017-10-22 08:00:00  0.027796  2.901876e+06  0.030174  0.027615   
...                              ...           ...       ...       ...   
xrp    2023-05-13 08:00:00  0.431633  1.316062e+09  0.434140  0.423749   
       2023-05-14 08:00:00  0.425562  4.013834e+08  0.431169  0.422264   
       2023-05-15 08:00:00  0.426351  3.8

In [477]:
# Print the analysis results for all groups
# Export the DataFrame to a CSV file
results.to_csv('results.csv')

# Print a message to confirm the export
print('The results DataFrame has been exported to results.csv.')

The results DataFrame has been exported to results.csv.
